In [ ]:
import math
import tensorflow as tf

In [ ]:
digits = tf.ragged.constant([[3,1,3,4], [], [5,9,2], [6], []])
words = tf.ragged.constant([['So', 'long'], ['thanks', 'for', 'all', 'the', 'fish']])

print(tf.add(digits, 3))

In [ ]:
print(tf.reduce_mean(digits, axis=1))

In [ ]:
print(tf.concat([digits, [[5,3]]], axis=0))

In [ ]:
print(tf.tile(digits, [1, 2]))

In [ ]:
print(tf.tile(digits, [2, 1]))

In [ ]:
print(tf.strings.substr(words, 0, 2))

In [ ]:
print(tf.map_fn(tf.math.square, digits))

In [ ]:
print(digits[0])

In [ ]:
print(digits[:, :2])

In [ ]:
print(digits[:, -2:])

In [ ]:
print(digits + 3)

In [ ]:
print(tf.ragged.map_flat_values(lambda x: x*2 + 1, digits))

In [ ]:
digits.to_list()

In [ ]:
digits.numpy()

In [ ]:
sentences = tf.ragged.constant([
    ["Let's", "build", "some", "ragged", "tensors", "!"],
    ["We", "can", "use", "tf.ragged.constant", "."],
])

print(sentences)

In [ ]:
paragraphs = tf.ragged.constant([
    [['I', 'have', 'a', 'cat'], ['His', 'name', 'is', 'Mat']],
    [['Do', 'you', 'want', 'to', 'come', 'visit'], ["I'm", 'free', 'tomorrow']],
])
print(paragraphs)

In [ ]:
print(tf.RaggedTensor.from_value_rowids(
    values = [3,1,4,1,5,9,2],
    value_rowids = [0,0,0,0,2,2,3]
))

In [ ]:
print(tf.RaggedTensor.from_row_lengths(
    values=[3,1,4,1,5,9,2],
    row_lengths=[4,0,2,1]
))

In [ ]:
print(tf.RaggedTensor.from_row_splits(
    values=[3,1,4,1,5,9,2],
    row_splits=[0,4,4,6,7]
))

In [ ]:
print(tf.ragged.constant([['Hi!'], ['How', 'are', 'you']]))

In [ ]:
print(tf.ragged.constant([[[1, 2], [3]], [[4, 5]]]))

In [ ]:
try:
    tf.ragged.constant([['one', 'two'], [3, 4]])
except ValueError as e:
    print(e)

In [ ]:
try:
    tf.ragged.constant(["A", ["B", "C"]])
except ValueError as e:
    print(e)

In [ ]:
queries = tf.ragged.constant([
    ['Who', 'is', 'Dan', 'Smith'],
    ['Pause'],
    ['Will', 'it', 'rain', 'later', 'day'],
])

In [ ]:
num_buckets = 1024
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal(
        [num_buckets, embedding_size], stddev = 1.0 / math.sqrt(embedding_size)
    )
)

In [ ]:
word_buckets = tf.strings.to_hash_bucket_fast(queries, num_buckets)
word_buckets

In [ ]:
word_embeddings = tf.nn.embedding_lookup(embedding_table, word_buckets)
word_embeddings.shape

In [ ]:
marker = tf.fill([queries.nrows(), 1], '#')
marker

In [ ]:
padded = tf.concat([marker, queries, marker], axis=1)
print(padded)

In [ ]:
bigrams = tf.strings.join([padded[:, :-1], padded[:, 1:]], separator='+')
bigrams

In [ ]:
bigram_buckets = tf.strings.to_hash_bucket_fast(bigrams, num_buckets)
bigram_buckets

In [ ]:
bigram_embeddings = tf.nn.embedding_lookup(embedding_table, bigram_buckets)
bigram_embeddings

In [ ]:
all_embeddings = tf.concat([word_embeddings, bigram_embeddings], axis=1)
all_embeddings.shape

In [ ]:
avg_embedding = tf.reduce_mean(all_embeddings, axis=1)
avg_embedding

In [ ]:
tf.ragged.constant([["Hi"], ["How", "are", "you"]]).shape

In [ ]:
print(tf.ragged.constant([['Hi'], ['How', 'are', 'you']]).bounding_shape())

In [ ]:
ragged_x = tf.ragged.constant([['John'], ['a', 'big', 'dog'], ['my', 'cat']])
ragged_y = tf.ragged.constant([['fell', 'asleep'], ['barked'], ['is', 'fuzzy']])
print(tf.concat([ragged_x, ragged_y], axis=1))

In [ ]:
sparse_x = ragged_x.to_sparse()
sparse_y = ragged_y.to_sparse()
sparse_result = tf.sparse.concat(sp_inputs=[sparse_x, sparse_y], axis=1)
print(tf.sparse.to_dense(sparse_result, ' '))

In [ ]:
sentences = tf.constant([
    'What makes you think she is a witch?',
    'She turned me into a newt.',
    'A newt?',
    'Well, I got better.'
])

is_question = tf.constant([True, False, True, False])

hash_buckets = 1000
words = tf.strings.split(sentences, ' ')
hashed_words = tf.strings.to_hash_bucket_fast(words, hash_buckets)

keras_model = tf.keras.Sequential([
    tf.keras.layers.Input([None], dtype=tf.int64, ragged=True),
    tf.keras.layers.Embedding(hash_buckets, 16),
    tf.keras.layers.LSTM(32, use_bias=False),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

keras_model.compile(loss='binary_crossentropy', optimizer='rmsprop')
keras_model.fit(hashed_words, is_question, epochs=5)
print(keras_model.predict(hashed_words))

In [ ]:
import google.protobuf.text_format as pbtext

def build_tf_example(s):
    return pbtext.Merge(s, tf.train.Example()).SerializeToString()

In [ ]:
example_batch = [
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["red", "blue"]} } }
      feature {key: "lengths" value {int64_list {value: [7]} } } }'''),
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["orange"]} } }
      feature {key: "lengths" value {int64_list {value: []} } } }'''),
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["black", "yellow"]} } }
      feature {key: "lengths" value {int64_list {value: [1, 3]} } } }'''),
  build_tf_example(r'''
    features {
      feature {key: "colors" value {bytes_list {value: ["green"]} } }
      feature {key: "lengths" value {int64_list {value: [3, 5, 2]} } } }''')]

In [ ]:
feature_specification = {
    'colors': tf.io.RaggedFeature(tf.string),
    'lengths': tf.io.RaggedFeature(tf.int64),
}

feature_tensors = tf.io.parse_example(example_batch, feature_specification)
for name, value in feature_tensors.items():
    print('{}={}'.format(name, value))

In [ ]:
def print_dictionary_dataset(dataset):
    for i, element in enumerate(dataset):
        print('Element {}:'.format(i))
        for (name, value) in element.items():
            print('{:>14} = {}'.format(name, value))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(feature_tensors)
print_dictionary_dataset(dataset)

In [ ]:
batch_dataset = dataset.batch(2)
print_dictionary_dataset(batch_dataset)

In [ ]:
unbatch_dataset = batch_dataset.unbatch()
print_dictionary_dataset(unbatch_dataset)

In [ ]:
non_ragged_dataset = tf.data.Dataset.from_tensor_slices([1, 5, 3, 2, 8])
non_ragged_dataset = non_ragged_dataset.map(tf.range)
batched_non_ragged_dataset = non_ragged_dataset.apply(tf.data.experimental.dense_to_ragged_batch(2))
for element in batched_non_ragged_dataset:
    print(element)

In [ ]:
def transform_lengths(features):
    return {
        'mean_length': tf.math.reduce_mean(features['lengths']),
        'length_ranges': tf.ragged.range(features['lengths'])
    }

transformed_dataset = dataset.map(transform_lengths)
print_dictionary_dataset(transformed_dataset)

In [ ]:
@tf.function
def make_palindrome(x, axis):
    return tf.concat([x, tf.reverse(x, [axis])], axis)

In [ ]:
make_palindrome(x=tf.constant([[1, 2], [3, 4], [5, 6]]), axis=1)

In [ ]:
make_palindrome(x=tf.ragged.constant([[1, 2], [3], [4, 5, 6]]), axis=1)

In [ ]:
@tf.function(input_signature=[tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int32)])
def max_and_min(rt):
    return (tf.math.reduce_max(rt, axis=-1), tf.math.reduce_min(rt, axis=-1))

max_and_min(tf.ragged.constant([[1, 2], [3], [4, 5, 6]]))

In [ ]:
@tf.function
def increment(x):
    return x+1

rt = tf.ragged.constant([[1, 2], [3], [4, 5, 6]])
cf = increment.get_concrete_function(rt)
print(cf(rt))

In [ ]:
import tempfile

keras_module_path = tempfile.mkdtemp()
tf.saved_model.save(keras_model, keras_module_path)
imported_model = tf.saved_model.load(keras_module_path)
imported_model(hashed_words)

In [ ]:
class CustomModule(tf.Module):
    def __init__(self, variable_value):
        super(CustomModule, self).__init__()
        self.v = tf.Variable(variable_value)
    
    @tf.function
    def grow(self, x):
        return x * self.v

module = CustomModule(100.0)
module.grow.get_concrete_function(tf.RaggedTensorSpec(shape=[None, None], dtype=tf.float32))

custom_module_path = tempfile.mkdtemp()
tf.saved_model.save(module, custom_module_path)
imported_model = tf.saved_model.load(custom_module_path)

imported_model.grow(tf.ragged.constant([[1.0, 2.0], [3.0], [4.0, 5.0, 6.0]]))

In [ ]:
x = tf.ragged.constant([[1, 2], [3], [4, 5, 6]])
y = tf.ragged.constant([[1, 1], [2], [3, 3, 3]])
x + y

In [ ]:
x + 3

In [ ]:
queries = tf.ragged.constant(
    [['Who', 'is', 'George', 'Washington'],
     ['What', 'is', 'the', 'weather', 'tomorrow'],
     ['Goodnight']])

In [ ]:
queries[1]

In [ ]:
print(queries[1, 2])

In [ ]:
print(queries[1:])

In [ ]:
print(queries[:, :3])

In [ ]:
print(queries[:, -2:])

In [ ]:
rt = tf.ragged.constant([[[1, 2, 3], [4]],
                         [[5], [], [6]],
                         [[7]],
                         [[8, 9], [10]]])

In [ ]:
print(rt[1])

In [ ]:
print(rt[3, 0])

In [ ]:
print(rt[:, 1:3])

In [ ]:
print(rt[:, -1:])

In [ ]:
ragged_sentences = tf.ragged.constant([
    ['Hi'], ['Welcome', 'to', 'the', 'fair'], ['Have', 'fun']])

In [ ]:
print(ragged_sentences.to_tensor(default_value=' '))

In [ ]:
x = [[1, 3, -1, -1], [2, -1, -1, -1], [4, 5, 8, 9]]
print(tf.RaggedTensor.from_tensor(x, padding=-1))

In [ ]:
print(ragged_sentences.to_sparse())

In [ ]:
st = tf.SparseTensor(indices=[[0,0], [2,0], [2,1]], values=['a', 'b', 'c'], dense_shape=[3,3])
print(tf.RaggedTensor.from_sparse(st))

In [ ]:
rt = tf.ragged.constant([[1, 2], [3, 4, 5], [6], [], [7]])

In [ ]:
print('python list: ', rt.to_list())

In [ ]:
print('numpy array: ', rt.numpy())

In [ ]:
print('values: ', rt.values.numpy())

In [ ]:
print('splits: ', rt.row_splits.numpy())

In [ ]:
print('index value: ', rt[1].numpy())

In [ ]:
x = tf.ragged.constant([[1, 2], [3]])
y = 3

x + y

In [ ]:
x = tf.ragged.constant(
   [[10, 87, 12],
    [19, 53],
    [12, 32]])
y = [[1000], [2000], [3000]]
print(x + y)

In [ ]:
x = tf.ragged.constant(
    [[[1, 2], [3, 4], [5, 6]],
     [[7, 8]]],
    ragged_rank=1)
y = tf.constant([[10]])
print(x + y)

In [ ]:
x = tf.ragged.constant(
    [
        [
            [[1], [2]],
            [],
            [[3]],
            [[4]],
        ],
        [
            [[5], [6]],
            [[7]]
        ]
    ],
    ragged_rank=2)
y = tf.constant([10, 20, 30])
print(x + y)

In [ ]:
x = tf.ragged.constant([[1, 2], [3, 4, 5, 6], [7]])
y = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

In [ ]:
x = tf.ragged.constant([[1, 2, 3], [4], [5, 6]])
y = tf.ragged.constant([[10, 20], [30, 40], [50]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

In [ ]:
x = tf.ragged.constant([[[1, 2], [3, 4], [5, 6]],
                        [[7, 8], [9, 10]]])
y = tf.ragged.constant([[[1, 2, 0], [3, 4, 0], [5, 6, 0]],
                        [[7, 8, 0], [9, 10, 0]]])
try:
  x + y
except tf.errors.InvalidArgumentError as exception:
  print(exception)

In [ ]:
rt = tf.RaggedTensor.from_row_splits(
    values=[3, 1, 4, 1, 5, 9, 2],
    row_splits=[0, 4, 4, 6, 7])
print(rt)

In [ ]:
rt = tf.RaggedTensor.from_row_splits(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 3, 5, 9, 10]
    ),
    row_splits=[0, 1, 1, 5]
)

print(rt)
print("Shape: ", rt.shape)
print("Number of partitioned dimensions: ", rt.ragged_rank)

In [ ]:
rt = tf.RaggedTensor.from_nested_row_splits(
    flat_values = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    nested_row_splits = ([0, 1, 1, 5], [0, 3, 3, 5, 9, 10])
)

print(rt)

In [ ]:
conversations = tf.ragged.constant(
    [[[["I", "like", "ragged", "tensors."]],
      [["Oh", "yeah?"], ["What", "can", "you", "use", "them", "for?"]],
      [["Processing", "variable", "length", "data!"]]],
     [[["I", "like", "cheese."], ["Do", "you?"]],
      [["Yes."], ["I", "do."]]]])

conversations.shape

In [ ]:
conversations.ragged_rank

In [ ]:
conversations.nested_row_splits

In [ ]:
conversations.flat_values.numpy()

In [ ]:
rt = tf.RaggedTensor.from_row_splits(
    values=[[1, 3], [0, 0], [1, 3], [5, 3], [3, 3], [1, 2]],
    row_splits=[0, 3, 4, 6])
print(rt)
print("Shape: {}".format(rt.shape))
print("Number of partitioned dimensions: {}".format(rt.ragged_rank))
print("Flat values shape: {}".format(rt.flat_values.shape))
print("Flat values:\n{}".format(rt.flat_values))

In [ ]:
rt = tf.RaggedTensor.from_uniform_row_length(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 5, 9, 10]),
    uniform_row_length=2)
print(rt)
print("Shape: {}".format(rt.shape))
print("Number of partitioned dimensions: {}".format(rt.ragged_rank))